# Submission : Proyek Pengembangan Machine Learning Pipeline

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import pandas as pd

## Set Variable

In [2]:
PIPELINE_NAME = "apple-quality-pipeline"
SCHEMA_PIPELINE_NAME = "apple-quality-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

Cek isi dari dataset apple quality. Pipeline ini akan menggunakan data yang sudah tersebut untuk memprediksi kualitas apel.

In [88]:
df = pd.read_csv(r'D:\Eldon\Belajar\Bangkit\MLOps - Dicoding\MLOps-Tugas1\data\apple_quality.csv')
df = df.drop('A_id', axis=1)
df = df.dropna()
df['Acidity'] = df['Acidity'].astype(float)
df['Quality'] = df['Quality'].map({'good': 1, 'bad': 0})
df.to_csv(r'D:\Eldon\Belajar\Bangkit\MLOps - Dicoding\MLOps-Tugas1\data\apple_quality.csv', index=False)

Dari data tersebut, ditetapkan lokasi dataset yang akan digunakan pada pipeline.

In [90]:
DATA_ROOT = "data"

In [91]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Tahapan Data Ingestion

Kemudian pada kode di bawah digunakan pipeline ExampleGen untuk mengambil data dan membagi data tersebut mejadi data train-eval dengan ratio 8:2.

In [92]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [93]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 65
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

# Tahapan Data Validation

Kemudian setelah melakukan ingestion, dilakukan data validation. Disini digunakan StatisticGen dan SchemaGen untuk melihat beberapa informasi mengenai data dan skemanya.

In [94]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 66
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [95]:
interactive_context.show(statistics_gen.outputs["statistics"])

In [96]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 67
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [97]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Acidity',FLOAT,required,,-
'Crunchiness',FLOAT,required,,-
'Juiciness',FLOAT,required,,-
'Quality',INT,required,,-
'Ripeness',FLOAT,required,,-
'Size',FLOAT,required,,-
'Sweetness',FLOAT,required,,-
'Weight',FLOAT,required,,-


# Mengidentifikasi anomali pada dataset

Kemudian kita akan mengecek dataset yang dibentuk apakah terdapat anomali yang dapat mempengaruhi performa model menggunakan pipeline ExampleValidator

In [103]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 70
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [104]:
interactive_context.show(example_validator.outputs['anomalies'])

# Tahapan Data Preprocessing

Data yang diperoleh kemudian akan dilakukan transform atau preprocessing data sehingga data dapat digunakan untuk proses modeling machine learning

In [105]:
TRANSFORM_MODULE_FILE = "applequality_transform.py"

In [106]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf

LABEL_KEY = "Quality"
FEATURE_KEY = ['Acidity', 'Crunchiness', 'Juiciness', 'Ripeness', 'Size', 'Sweetness', 'Weight']

def _transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """Preprocess input features."""
    outputs = {}

    # Normalize numerical features
    for feature_name in FEATURE_KEY:
        outputs[_transformed_name(feature_name)] = inputs[feature_name]

    # Leave 'quality' as it is (treat it as a categorical feature)
    outputs[_transformed_name('Quality')] = inputs['Quality']

    return outputs

Overwriting applequality_transform.py


In [107]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Transform\transform_graph\71\.temp_path\tftransform_tmp\e2ff8a673f824a7082160009d7811f17\assets


INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Transform\transform_graph\71\.temp_path\tftransform_tmp\e2ff8a673f824a7082160009d7811f17\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 71
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

# Tahapan Pengembangan Model

In [108]:
TRAINER_MODULE_FILE = "applequality_trainer.py"

In [111]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "Quality"
FEATURE_KEY = ["Acidity", "Crunchiness", "Juiciness", "Ripeness", "Size", "Sweetness", "Weight"]

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64) -> tf.data.Dataset:
    """Get post_transform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    return dataset.repeat()

def model_builder():
    """Build machine learning model"""
    inputs = [
        tf.keras.Input(shape=(1,), name=transformed_name(f), dtype=tf.float64)
        for f in [
            "Acidity", "Crunchiness", "Juiciness", "Ripeness", "Size", "Sweetness", "Weight"
        ]
    ]
    concatenated_features = tf.keras.layers.concatenate(inputs)
    x = layers.Dense(64, activation='relu')(concatenated_features)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    model.summary()
    return model 

def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # Get predictions using the transformed features
        return model(transformed_features)
    
        # Sanitize input names for SavedModel signature
        sanitized_input_names = [f.replace(' ', '_').lower() + '_xf' for f in feature_spec.keys()]
    
        serving_default_signature = serve_tf_examples_fn.get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'
            )
        )
        serving_default_signature._input_signature[0].name = 'examples'  # Rename input tensor
    
        # Assign sanitized names to input tensors
        for i, input_tensor in enumerate(serving_default_signature.inputs):
            input_tensor._name = sanitized_input_names[i]  # Assign sanitized name to input tensor

    return serve_tf_examples_fn




def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_loss', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)

    model = model_builder()

    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000, 
            validation_steps= 1000,
            epochs=10)
            
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'
            )
        )
    }
    
    # Save the model with signatures
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting applequality_trainer.py


In [112]:
from tfx.proto import trainer_pb2
 
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Acidity_xf (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 Crunchiness_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 Juiciness_xf (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 Ripeness_xf (InputLayer)       [(None, 1)]          0           []                               
                                                                                            

INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\73\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\73\Format-Serving\assets


1000/1000 [==============================] - 7s 6ms/step - loss: 0.2829 - accuracy: 0.8784 - val_loss: 0.2373 - val_accuracy: 0.9065
Epoch 2/10
 991/1000 [============================>.] - ETA: 0s - loss: 0.1664 - accuracy: 0.9360
Epoch 2: val_loss did not improve from 0.23729
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1662 - accuracy: 0.9360 - val_loss: 0.2136 - val_accuracy: 0.9177
Epoch 3/10
 985/1000 [============================>.] - ETA: 0s - loss: 0.1192 - accuracy: 0.9568
Epoch 3: val_loss did not improve from 0.23729
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1190 - accuracy: 0.9569 - val_loss: 0.2061 - val_accuracy: 0.9303
Epoch 4/10
1000/1000 [==============================] - ETA: 0s - loss: 0.0932 - accuracy: 0.9670
Epoch 4: val_loss did not improve from 0.23729
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0932 - accuracy: 0.9670 - val_loss: 0.2129 - val_accuracy: 0.9341
Epoch 5/10
 982/1000 [===========

INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\73\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\73\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\73\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 73
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

# Tahapan Tuner

In [113]:
TUNER_MODULE_FILE = "applequality_tuner.py"

In [121]:
%%writefile {TUNER_MODULE_FILE}
from typing import NamedTuple, Dict, Any, Text
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from keras_tuner.engine import base_tuner
from keras_tuner.tuners import Hyperband
from keras_tuner.engine.base_tuner import BaseTuner
import tensorflow_transform as tft
import tensorflow as tf

TunerFnResult = NamedTuple('TunerFnResult', [('tuner', BaseTuner), ('fit_kwargs', Dict[Text, Any])])

LABEL_KEY = 'Quality'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs=None,
             batch_size=64) -> tf.data.Dataset:
    """Get post_transform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    return dataset.repeat()
    
def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """Build the tuner using the KerasTuner API.

    Args:
        fn_args: Holds args used to tune models as name/value pairs.

    Returns:
        A TunerFnResult that contains the tuner and fit_kwargs.
    """
    # Load the transformed data
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files[0], tf_transform_output)
    val_set = input_fn(fn_args.eval_files[0], tf_transform_output)

    def model_builder(hp):
        hp_units = hp.Int('units', min_value=128, max_value=512, step=32)
        hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])
        """Build machine learning model"""
        inputs = [
            tf.keras.Input(shape=(1,), name=transformed_name(f), dtype=tf.float64)
            for f in [
                "Acidity", "Crunchiness", "Juiciness", "Ripeness", "Size", "Sweetness", "Weight"
            ]
        ]
        concatenated_features = tf.keras.layers.concatenate(inputs)
        x = layers.Dense(64, activation='relu')(concatenated_features)
        x = layers.Dense(32, activation='relu')(x)
        outputs = layers.Dense(1, activation='sigmoid')(x)
    
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
        model.compile(
            loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        
        return model
    
    tuner = Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory=fn_args.working_dir,
                     project_name='kt_hyperband')
        
    early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.001, patience=5, verbose=1)
    
    fit_kwargs = {
        "callbacks": [early_stopping],
        'x': train_set,
        'validation_data': val_set,
        'steps_per_epoch': fn_args.train_steps,
        'validation_steps': fn_args.eval_steps
    }

    return TunerFnResult(tuner=tuner, fit_kwargs=fit_kwargs)

Overwriting applequality_tuner.py


In [122]:
tuner = Tuner(
    module_file=TUNER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=500),  # Adjust as needed
    eval_args=trainer_pb2.EvalArgs(num_steps=100)  # Adjust as needed
)

# Run the TFX pipeline with the Tuner component
interactive_context.run(tuner)

Trial 30 Complete [00h 00m 21s]
val_accuracy: 0.9307812452316284

Best val_accuracy So Far: 0.9390624761581421
Total elapsed time: 00h 04m 38s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\apple-quality-pipeline\.temp\77\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
units: 160
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9390624761581421
Trial summary
Hyperparameters:
units: 128
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.938281238079071
Trial summary
Hyperparameters:
units: 288
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9340624809265137
Trial summary
Hyperparameters:
units: 288
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9307812452316284
Trial summary
Hyperparameters:
units: 256
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0021
Score: 0.9278125166893005
Trial summary
Hyperparameters:
units: 192
learning

ExecutionResult(
    component_id: Tuner
    execution_id: 77
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

In [123]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=TRAINER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=100)
)

interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Acidity_xf (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 Crunchiness_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 Juiciness_xf (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 Ripeness_xf (InputLayer)       [(None, 1)]          0           []                               
                                                                                            

INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\78\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\78\Format-Serving\assets


1000/1000 [==============================] - 10s 9ms/step - loss: 0.2909 - accuracy: 0.8735 - val_loss: 0.2475 - val_accuracy: 0.8939
Epoch 2/10
 982/1000 [============================>.] - ETA: 0s - loss: 0.1819 - accuracy: 0.9290
Epoch 2: val_loss did not improve from 0.24753
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1811 - accuracy: 0.9294 - val_loss: 0.2233 - val_accuracy: 0.9152
Epoch 3/10
1000/1000 [==============================] - ETA: 0s - loss: 0.1386 - accuracy: 0.9488
Epoch 3: val_loss did not improve from 0.24753
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1386 - accuracy: 0.9488 - val_loss: 0.2209 - val_accuracy: 0.9166
Epoch 4/10
 987/1000 [============================>.] - ETA: 0s - loss: 0.1143 - accuracy: 0.9585
Epoch 4: val_loss did not improve from 0.24753
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1142 - accuracy: 0.9586 - val_loss: 0.2288 - val_accuracy: 0.9179
Epoch 5/10
 996/1000 [==========

INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\78\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\78\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\apple-quality-pipeline\Trainer\model\78\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 78
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [124]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 79
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [127]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Quality')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [128]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 81
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [129]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

In [130]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/apple-quality-model'))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 82
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))